In [8]:
#Example 1: 

import numpy as np
def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for Cubic symmetry
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            print(f"Angle between Axis and CSL Axis {perm:}: {angle:.2f}°")
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=10.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                
                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('abc')))
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Example usage
axis1 = [17, 17, -16]
misorientation_angle1 = 59.6
structure_type = 'FCC'  # 'FCC' or 'BCC'

is_sigma1, sigma_value1, deviation1 = is_sigma_boundary(axis1, misorientation_angle1, structure_type)
if is_sigma1:
    print(f"This is a Σ{sigma_value1} boundary with a deviation of {deviation1:.2f} degrees")
else:
    print("This is not a Σ boundary")

axis2 = [16, 16, -16]
misorientation_angle2 = 60

is_sigma2, sigma_value2, deviation2 = is_sigma_boundary(axis2, misorientation_angle2, structure_type)
if is_sigma2:
    print(f"This is a Σ{sigma_value2} boundary with a deviation of {deviation2:.2f} degrees")
else:
    print("This is not a Σ boundary")

    
axis3 = [0.1142, 0.0523, 0.9921]
misorientation_angle3 = 19.24199995
is_sigma3, sigma_value3, deviation3 = is_sigma_boundary(axis3, misorientation_angle3, structure_type)
if is_sigma3:
    print(f"This is a Σ{sigma_value3} boundary with a deviation of {deviation3:.2f} degrees")
else:
    print("This is not a Σ boundary")
    


Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 1.62°
This is a Σ3 boundary with a deviation of 0.40 degrees
Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 0.00°
This is a Σ3 boundary with a deviation of 0.00 degrees
Angle between Axis and CSL Axis [0.0, 0.0, 1.0]: 7.22°
Angle between Axis and CSL Axis [0.0, 0.0, 1.0]: 7.22°
This is a Σ13a boundary with a deviation of 3.38 degrees


In [12]:
#Example 2: 

import numpy as np

def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for symmetry of Cubic system
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            #print(f"Angle between Axis and CSL Axis {perm}: {angle:.2f}°")
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=5.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                
                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('abc')))
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Given data
misori_angle = [39.99032809, 34.61751482, 31.8568031, 33.75154261, 47.8394252, 52.18859909, 59.6111677, 45.55147369, 19.51118361, 34.19565999, 28.66987262, 52.3075686, 29.70978892]

Axis = np.array([
    [0.2875, 0.6517, 0.7019],
    [0.4314, 0.0756, 0.899],
    [0.7027, 0.0415, 0.7103],
    [0.1249, 0.0627, 0.9902],
    [0.2368, 0.4618, 0.8548],
    [0.2609, 0.4685, 0.8441],
    [0.5721, 0.5795, 0.5804],
    [0.1318, 0.5213, 0.8432],
    [0.3954, 0.2071, 0.8948],
    [0.0288, 0.1988, 0.9796],
    [0.39, 0.3213, 0.8629],
    [0.6881, 0.1837, 0.702],
    [0.6183, 0.4417, 0.65],
])

# Check each axis and misorientation angle
results = []
for axis, angle in zip(Axis, misori_angle):
    is_sigma, sigma_value, deviation = is_sigma_boundary(axis, angle, structure_type='BCC')
    if is_sigma:
        results.append((axis, angle, sigma_value, deviation))
        print(f"Axis: {axis}, Misorientation Angle: {angle:.2f}°, Σ Boundary: {sigma_value}, Deviation: {deviation:.2f}°")
    else:
        results.append((axis, angle, None, None))
        print(f"Axis: {axis}, Misorientation Angle: {angle:.2f}°, Not a Σ Boundary")



Axis: [0.2875 0.6517 0.7019], Misorientation Angle: 39.99°, Not a Σ Boundary
Axis: [0.4314 0.0756 0.899 ], Misorientation Angle: 34.62°, Σ Boundary: 27b, Deviation: 0.81°
Axis: [0.7027 0.0415 0.7103], Misorientation Angle: 31.86°, Σ Boundary: 27a, Deviation: 0.27°
Axis: [0.1249 0.0627 0.9902], Misorientation Angle: 33.75°, Not a Σ Boundary
Axis: [0.2368 0.4618 0.8548], Misorientation Angle: 47.84°, Not a Σ Boundary
Axis: [0.2609 0.4685 0.8441], Misorientation Angle: 52.19°, Σ Boundary: 39b, Deviation: 2.09°
Axis: [0.5721 0.5795 0.5804], Misorientation Angle: 59.61°, Σ Boundary: 3, Deviation: 0.39°
Axis: [0.1318 0.5213 0.8432], Misorientation Angle: 45.55°, Not a Σ Boundary
Axis: [0.3954 0.2071 0.8948], Misorientation Angle: 19.51°, Not a Σ Boundary
Axis: [0.0288 0.1988 0.9796], Misorientation Angle: 34.20°, Not a Σ Boundary
Axis: [0.39   0.3213 0.8629], Misorientation Angle: 28.67°, Not a Σ Boundary
Axis: [0.6881 0.1837 0.702 ], Misorientation Angle: 52.31°, Σ Boundary: 25b, Deviation:

In [6]:
#Example 3: By two orientations of grains 
#For obtaining rotation axis and misorientation angle
from orix.quaternion import Orientation, symmetry, Rotation
from orix.vector import Vector3d
import numpy as np
from scipy.spatial.transform import Rotation as R

def misorientation_calc(euler_grain1,euler_grain2):
    o1 = Orientation.from_euler(
        np.deg2rad(euler_grain1), symmetry=symmetry.Oh
    )
    print(o1)
    o2 = Orientation.from_euler(
        np.deg2rad(euler_grain2), symmetry=symmetry.Oh
    )
    print(o2)
    m = o1 - o2
    print(m)
    print(
        "Axis:",
        m.axis,
        "Misori. angle:",
        np.rad2deg(m.angle),
    )
    return  m.axis, np.rad2deg(m.angle)


def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for symmetry of Cubic system
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            #print(f"Angle between Axis and CSL Axis {perm}: {angle:.2f}°")
            return True, perm
    
    return False, None

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=5.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold)[0]:
                predefined_angle = config['angle']
                standard_axis=are_axes_equivalent(axis, predefined_axis, angle_threshold)[1]

                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('abc')))
                
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation, standard_axis, predefined_angle
    
    return False, None, None, None, None


#euler_grain1 = [78,133.2,232.6]
#euler_grain2 = [275,143.2,256.7]
#euler_grain1 = [0,0,0]
#euler_grain2=[340.7477,101.71763912,70.74769653]
euler_grain1 = [4.3,67.2,271]
euler_grain2=[332.2,107.0,93.9]

axis, misorientation_angle =misorientation_calc(euler_grain1,euler_grain2)
crystal_axis = axis[0].data.flatten()

is_sigma, sigma_value, deviation, standard_axis, predefined_angle = is_sigma_boundary(crystal_axis, misorientation_angle[0], structure_type='BCC')
if is_sigma:
    print(f"This is a Σ{sigma_value} boundary with a deviation of {deviation:.2f} degrees")
    print(f"Σ{sigma_value} boundary")
    print(f"standard_axis：{standard_axis}, angle：{predefined_angle:.2f}")
else:
    print("This is not a Σ boundary")




Orientation (1,) m-3m
[[ 0.6156 -0.3799 -0.4024  0.5611]]
Orientation (1,) m-3m
[[ 0.4986 -0.3916  0.702  -0.3244]]
Misorientation (1,) m-3m, m-3m
[[0.9037 0.0089 0.2899 0.315 ]]
Axis: Vector3d (1,)
[[0.0207 0.677  0.7357]] Misori. angle: [50.70220194]
This is a Σ11 boundary with a deviation of 0.22 degrees
Σ11 boundary
standard_axis：[0.0, 0.7071067811865475, 0.7071067811865475], angle：50.48


In [10]:
##Example 4: By two orientations of grains , The Devation calculation is more accurate here
#For obtaining rotation axis and misorientation angle
from orix.quaternion import Orientation, Quaternion, symmetry
from orix.vector import Vector3d
import numpy as np
from scipy.spatial.transform import Rotation as R


def rotation_matrix_to_euler(rotation_matrix):
    """Convert a rotation matrix to Bunge Euler angles."""
    return rotation_matrix.as_euler('ZXZ', degrees=True)


def normalize_vector(vector):
    """Normalize a vector."""
    norm = np.linalg.norm(vector)
    if norm == 0:
        return vector
    return vector / norm


def misorientation_angle(rot1, rot2):
    """Calculate the misorientation angle between two rotation matrices."""
    delta_rot = rot1.inv() * rot2
    angle = delta_rot.magnitude()
    return np.degrees(angle)


def crystal_rotation(euler_angles, axis, angle):
    """Rotate a crystal given initial Euler angles, rotation axis, and rotation angle."""
    # Convert Bunge Euler angles to a rotation matrix
    initial_rotation = R.from_euler('ZXZ', euler_angles, degrees=True)

    # Normalize the rotation axis
    crystal_axis = normalize_vector(axis)

    # Transform the crystal axis to the sample reference frame
    transformed_axis = initial_rotation.apply(crystal_axis)

    # Convert the rotation angle to radians
    angle_radians = np.radians(angle)

    # Create the rotation matrix for the specified axis rotation
    axis_rotation = R.from_rotvec(angle_radians * transformed_axis)

    # Compute the new rotation matrix
    new_rotation = axis_rotation * initial_rotation

    # Get the new Bunge Euler angles
    new_euler_angles = rotation_matrix_to_euler(new_rotation)

    return new_euler_angles


def compute_symmetry_reduced_orientation(ori1, ori2, symmetry_str='Oh'):
    """Compute the symmetry reduced orientations of ori1 with the smallest disorientation angle to ori2."""

    ori1_quat = Quaternion.from_euler(ori1, degrees=True)
    ori1_quat.symmetry = getattr(symmetry, symmetry_str)
    ori2_quat = Quaternion.from_euler(ori2, degrees=True)
    ori2_quat.symmetry = getattr(symmetry, symmetry_str)
    symmetry_ori = ori1_quat.symmetry
    angles = []
    ori2_sym = symmetry_ori.outer(ori2_quat)
    misorientation = ori1_quat * ~ori2_sym
    for orien in misorientation:
        orim = Orientation(orien)
        angles.append(orim.angle)
    angles_array = np.array(angles)
    indices = angles_array.argmin(axis=0)  # Minimum angle down symmetry dim.
    # Index one symmetry orientation for each initial orientation
    out = np.take_along_axis(ori2_sym, indices[np.newaxis], axis=0).squeeze()
    ori2_sym_reduced = out.to_euler(degrees=True)
    return ori2_sym_reduced


def misorientation(rot1, rot2):
    """Calculate the misorientation angle between two rotation matrices."""
    delta_rot = rot1.inv() * rot2
    misorientation_axis = delta_rot.as_rotvec()
    angle = np.degrees(delta_rot.magnitude())
    if 180 - angle < angle:
        misorientation_axis = -misorientation_axis
        angle = 180 - angle
    return misorientation_axis, angle


def misorientation_calc(euler_angles1, euler_angles2):
    """Calculate the misorientation axis and angle between two sets of Euler angles."""
    rot1 = R.from_euler('ZXZ', euler_angles1, degrees=True)
    rot2 = R.from_euler('ZXZ', euler_angles2, degrees=True)
    misorientation_axis, misorientation_ang = misorientation(rot1, rot2)
    return misorientation_axis, misorientation_ang


def misorientation_calc2(euler_grain1, euler_grain2):
    """Calculate the misorientation using orix's Orientation class."""
    o1 = Orientation.from_euler(np.deg2rad(euler_grain1), symmetry=symmetry.Oh)
    o2 = Orientation.from_euler(np.deg2rad(euler_grain2), symmetry=symmetry.Oh)
    m = o1 - o2
    axis = m.axis[0].flatten()
    angle = np.rad2deg(m.angle[0])
    return axis[0].data.flatten(), angle


def are_axes_equivalent(axis2, axis1, angle_threshold=5.0):
    """Check if two axes are equivalent considering crystal symmetry and slight deviations."""
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)

    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]], [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]], [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]], [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]], [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]], [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]], [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]], [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]], [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]], [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]], [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]], [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]], [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]], [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]], [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]], [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]], [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]], [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]], [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]], [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]], [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]], [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]], [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]], [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]], [-axis1[2], -axis1[0], -axis1[1]],
    ]

    for perm in axis_permutations:
        cos_angle = np.dot(perm, axis2)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = np.degrees(np.arccos(cos_angle))

        if angle <= angle_threshold:
            return True, perm

    return False, None


def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=15.0, tolerance_widening=2):
    """Determine if the given rotation axis and misorientation angle correspond to a Σ boundary."""
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},        
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    
    axis = np.array(axis)

    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            if are_axes_equivalent(axis, predefined_axis, angle_threshold)[0]:
                predefined_angle = config['angle']
                standard_axis = are_axes_equivalent(axis, predefined_axis, angle_threshold)[1]
                deviation = abs(misorientation_angle - predefined_angle)
                brandon_threshold = tolerance_widening* 15 / np.sqrt(float(sigma.rstrip('abc')))
                if deviation <= brandon_threshold:
                    print(f"brandon_threshold: {brandon_threshold}")
                    return True, sigma, deviation, standard_axis, predefined_angle

    return False, None, None, None, None


# Sample input Euler angles for two grains
euler_grain1 = [181, 138, 101]
euler_grain2 = [187, 137, 94]

euler_grain1 = [306,37.6,59.7]#5
euler_grain2 = [58.0,30.9,45.0]#6


euler_grain1 = [159.7001, 34.6899, 239.2699]#5
euler_grain2 = [280.9955, 31.0826, 43.0217]#6


euler_grain1 = [315, 28, 2]#5 20
euler_grain2 = [128.0609, 27.6541, 275.807]#6

euler_grain1 = [274, 143.1, 255.5]
euler_grain2 = [77, 132.9, 231.7]

euler_grain1 = [294.57, 46.91, 40.1301] # 3 0
euler_grain2 = [74.349, 25.8538, 282.2126] # 4 0

euler_grain1 = [74.349,25.8538,282.2126]#4
euler_grain2 = [10.5895,15.393, 324.2001]#8



# Compute symmetry-reduced orientation
euler_grain2_sym_reduced = compute_symmetry_reduced_orientation(euler_grain1, euler_grain2)
print(f"Symmetry reduced grain2 Euler angle (degrees): {euler_grain2_sym_reduced[0]}")

# Calculate misorientation axis and angle
axis, misorientation_ang = misorientation_calc(euler_grain1, euler_grain2_sym_reduced)
print(f"Rotation axis: {axis[0]}, misorientation angle (degrees): {misorientation_ang}")

# Check if it is a Σ boundary
is_sigma, sigma_value, deviation, standard_axis, predefined_angle = is_sigma_boundary(axis[0], misorientation_ang, structure_type='BCC')
if is_sigma:
    print(f"This is a Σ{sigma_value} boundary")
    print(f"Σ{sigma_value} boundary —— Standard axis: {standard_axis}, standard misorientation angle (degrees): {predefined_angle:.2f}")
    
    # Calculate standard Euler angles for calculating deviation
    std_euler_angles = crystal_rotation(euler_grain1, standard_axis, predefined_angle)
    print(f"Standard grain Bunge Euler angles with CSL Σ{sigma_value} relationship to grain 1 (degrees): {std_euler_angles}")
    
    # Calculate misorientation between perfect CSL orientation and actual grain orientation
    misorientation_axis, misorientation_ang = misorientation_calc(euler_grain2_sym_reduced, std_euler_angles)
    print(f"Deviation to the standard grain with CSL Σ{sigma_value} relationship (degrees): {misorientation_ang}")
else:
    print("This is not a low Σ boundary")


Symmetry reduced grain2 Euler angle (degrees): [ 10.5895  15.393  324.2001]
Rotation axis: [ 0.22906151 -0.33705517 -0.32062715], misorientation angle (degrees): [29.70978892]
brandon_threshold: 11.338934190276817
This is a Σ7 boundary
Σ7 boundary —— Standard axis: [0.5773502691896258, -0.5773502691896258, -0.5773502691896258], standard misorientation angle (degrees): 38.21
Standard grain Bunge Euler angles with CSL Σ7 relationship to grain 1 (degrees): [ -7.76064303  21.92683219 -23.30865659]
Deviation to the standard grain with CSL Σ7 relationship (degrees): [10.01067698]


In [7]:
import tkinter as tk
from tkinter import ttk, messagebox
from orix.quaternion import Quaternion, symmetry
import numpy as np

def compute_symmetry_reduced_orientation(ori1, ori2):
    if not all(isinstance(o, Quaternion) for o in (ori1, ori2)):
        raise TypeError("ori1 and ori2 must be orix.quaternion.Quaternion.")

    if ori2.size != 1 or ori2.ndim != 1:
        raise ValueError("ori2 must have shape (1,).")

    if ori1.symmetry == ori2.symmetry:
        symm = ori1.symmetry
    else:
        symm = ori1.symmetry.outer(ori2.symmetry).unique()
    
    angles = []
    ori1_sym = symm.outer(ori1)
    misorientation = ori2 * ~ori1_sym
    
    for orien in misorientation:
        orim = Quaternion(orien)
        angles.append(orim.angle)
    
    angles_array = np.array(angles)
    indices = angles_array.argmin(axis=0)
    out = np.take_along_axis(ori1_sym, indices[np.newaxis], axis=0).squeeze()
    return ori1.__class__(out)

def euler_to_symmetry_equivalent(euler_angles, symmetry_type):
    final_rotation_quat = Quaternion.from_euler(euler_angles, degrees=True)
    final_rotation_quat.symmetry = getattr(symmetry, symmetry_type)
    
    ori_standard = Quaternion([1, 0, 0, 0])
    ori_standard.symmetry = getattr(symmetry, symmetry_type)
    
    final_rotation = compute_symmetry_reduced_orientation(final_rotation_quat, ori_standard)
    final_euler_angles = final_rotation.to_euler(degrees=True)
    return final_euler_angles

def on_calculate():
    try:
        euler_angles = np.array([float(angle) for angle in euler_entry.get().split()])
        if len(euler_angles) != 3:
            raise ValueError("Please enter three Euler angles separated by spaces.")
        symmetry_type = symmetry_combobox.get()
        equivalent_euler_angles = euler_to_symmetry_equivalent(euler_angles, symmetry_type)
        result_label.config(text=f"Mininum rotation equivalent Euler angles: {equivalent_euler_angles}")
    
        rotation_quat = Quaternion.from_euler(euler_angles, degrees=True)
        rotation_quat.symmetry = getattr(symmetry, symmetry_type)
        symmetry_ori = rotation_quat.symmetry
        ori_sym = symmetry_ori.outer(rotation_quat)
        
        textbox.delete(1.0, tk.END)
        for ori in ori_sym:
            euler_symmetry = ori.to_euler(degrees=True)
            textbox.insert(tk.END, f"{euler_symmetry}\n")
    
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Tkinter setup
root = tk.Tk()
root.title("Euler to Symmetry Equivalent")

mainframe = ttk.Frame(root, padding="10")
mainframe.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

ttk.Label(mainframe, text="Euler Angles (degrees):").grid(row=0, column=0, sticky=tk.E)
euler_entry = ttk.Entry(mainframe)
euler_entry.grid(row=0, column=1)

ttk.Label(mainframe, text="Symmetry Type:").grid(row=1, column=0, sticky=tk.E)

# Dynamically populate the Combobox with symmetry types
labels = [
    s for s in dir(symmetry)
    if isinstance(getattr(symmetry, s), symmetry.Symmetry)
    and not s.startswith("_")
]
symmetry_combobox = ttk.Combobox(mainframe, values=labels, state="readonly")
symmetry_combobox.grid(row=1, column=1)
# Set the default selection to "Oh" if available
if "Oh" in labels:
    symmetry_combobox.set("Oh")
else:
    symmetry_combobox.set(labels[0])
    
calculate_button = ttk.Button(mainframe, text="Calculate", command=on_calculate)
calculate_button.grid(row=2, column=0, columnspan=2, pady=10)

result_label = ttk.Label(mainframe, text="")
result_label.grid(row=3, column=0, columnspan=2)

textbox = tk.Text(mainframe, height=10, width=50)
textbox.grid(row=4, column=0, columnspan=2, pady=10)

root.mainloop()


In [5]:
import numpy as np
from orix.quaternion import Quaternion, symmetry

def euler_to_rotation_matrix(euler_angles):
    phi1, PHI, phi2 = np.radians(euler_angles)
    
    Rz_phi1 = np.array([
        [np.cos(phi1), -np.sin(phi1), 0],
        [np.sin(phi1), np.cos(phi1), 0],
        [0, 0, 1]
    ])
    
    Rx_PHI = np.array([
        [1, 0, 0],
        [0, np.cos(PHI), -np.sin(PHI)],
        [0, np.sin(PHI), np.cos(PHI)]
    ])
    
    Rz_phi2 = np.array([
        [np.cos(phi2), -np.sin(phi2), 0],
        [np.sin(phi2), np.cos(phi2), 0],
        [0, 0, 1]
    ])
    
    rotation_matrix = Rz_phi1 @ Rx_PHI @ Rz_phi2
    return rotation_matrix

def get_crystallographic_directions(euler_angles):
    rotation_matrix = euler_to_rotation_matrix(euler_angles)
    
    # Reference crystallographic directions
    z_direction = np.array([0, 0, 1])
    x_direction = np.array([1, 0, 0])
    y_direction = np.array([0, 1, 0])
    
    # Apply the rotation matrix
    z_crystal_direction = rotation_matrix @ z_direction
    x_crystal_direction = rotation_matrix @ x_direction
    y_crystal_direction = rotation_matrix @ y_direction
    
    return z_crystal_direction, x_crystal_direction, y_crystal_direction

# Example Euler angles in degrees
euler_angles = [226.2978, 47.9185, 109.7089]

z_crystal_direction, x_crystal_direction, y_crystal_direction = get_crystallographic_directions(euler_angles)
print("Crystallographic direction of Z axis:", z_crystal_direction)
print("Crystallographic direction of X axis:", x_crystal_direction)
print("Crystallographic direction of Y axis:", y_crystal_direction)


Crystallographic direction of Z axis: [-0.53656094  0.51278819  0.67018701]
Crystallographic direction of X axis: [ 0.68912578 -0.19210777  0.69871329]
Crystallographic direction of Y axis: [ 0.48704006  0.83674541 -0.25029803]


In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np

# For misorientation calculations
from orix.quaternion import Orientation, Quaternion, symmetry
from orix.vector import Vector3d
from scipy.spatial.transform import Rotation as R

###############################################################################
# 1. Define all your existing functions as they are, so we can re-use them.   #
###############################################################################

def rotation_matrix_to_euler(rotation_matrix):
    """Convert a rotation matrix to Bunge Euler angles."""
    return rotation_matrix.as_euler('ZXZ', degrees=True)

def normalize_vector(vector):
    """Normalize a vector."""
    norm = np.linalg.norm(vector)
    if norm == 0:
        return vector
    return vector / norm

def misorientation_angle(rot1, rot2):
    """Calculate the misorientation angle between two rotation matrices."""
    delta_rot = rot1.inv() * rot2
    angle = delta_rot.magnitude()
    return np.degrees(angle)

def crystal_rotation(euler_angles, axis, angle):
    """Rotate a crystal given initial Euler angles, rotation axis, and rotation angle."""
    # Convert Bunge Euler angles to a rotation matrix
    initial_rotation = R.from_euler('ZXZ', euler_angles, degrees=True)

    # Normalize the rotation axis
    crystal_axis = normalize_vector(axis)

    # Transform the crystal axis to the sample reference frame
    transformed_axis = initial_rotation.apply(crystal_axis)

    # Convert the rotation angle to radians
    angle_radians = np.radians(angle)

    # Create the rotation matrix for the specified axis rotation
    axis_rotation = R.from_rotvec(angle_radians * transformed_axis)

    # Compute the new rotation matrix
    new_rotation = axis_rotation * initial_rotation

    # Get the new Bunge Euler angles
    new_euler_angles = rotation_matrix_to_euler(new_rotation)

    return new_euler_angles

def compute_symmetry_reduced_orientation(ori1, ori2, symmetry_str='Oh'):
    """Compute the symmetry reduced orientations of ori1 with the smallest disorientation angle to ori2."""
    ori1_quat = Quaternion.from_euler(ori1, degrees=True)
    ori1_quat.symmetry = getattr(symmetry, symmetry_str)
    ori2_quat = Quaternion.from_euler(ori2, degrees=True)
    ori2_quat.symmetry = getattr(symmetry, symmetry_str)
    symmetry_ori = ori1_quat.symmetry

    angles = []
    ori2_sym = symmetry_ori.outer(ori2_quat)
    misorientation = ori1_quat * ~ori2_sym

    for orien in misorientation:
        orim = Orientation(orien)
        angles.append(orim.angle)

    angles_array = np.array(angles)
    indices = angles_array.argmin(axis=0)  # Minimum angle down symmetry dimension
    out = np.take_along_axis(ori2_sym, indices[np.newaxis], axis=0).squeeze()
    ori2_sym_reduced = out.to_euler(degrees=True)
    return ori2_sym_reduced

def misorientation(rot1, rot2):
    """Calculate the misorientation axis and angle between two rotation matrices."""
    delta_rot = rot1.inv() * rot2
    misorientation_axis = delta_rot.as_rotvec()
    angle = np.degrees(delta_rot.magnitude())
    if 180 - angle < angle:
        misorientation_axis = -misorientation_axis
        angle = 180 - angle
    return misorientation_axis, angle

def misorientation_calc(euler_angles1, euler_angles2):
    """Calculate the misorientation axis and angle between two sets of Euler angles (ZXZ)."""
    rot1 = R.from_euler('ZXZ', euler_angles1, degrees=True)
    rot2 = R.from_euler('ZXZ', euler_angles2, degrees=True)
    misorientation_axis, misorientation_ang = misorientation(rot1, rot2)
    return misorientation_axis, misorientation_ang

def misorientation_calc2(euler_grain1, euler_grain2):
    """Calculate the misorientation using orix's Orientation class."""
    o1 = Orientation.from_euler(np.deg2rad(euler_grain1), symmetry=symmetry.Oh)
    o2 = Orientation.from_euler(np.deg2rad(euler_grain2), symmetry=symmetry.Oh)
    m = o1 - o2
    axis = m.axis[0].flatten()
    angle = np.rad2deg(m.angle[0])
    return axis[0].data.flatten(), angle

def are_axes_equivalent(axis2, axis1, angle_threshold=5.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations.
    We generate permutations (including sign flips) of axis1 and compare to axis2.
    """
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)

    axis_permutations = []
    signs = [+1, -1]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if len({i, j, k}) == 3: # all different indices
                    for s1 in signs:
                        for s2 in signs:
                            for s3 in signs:
                                perm = [s1*axis1[i], s2*axis1[j], s3*axis1[k]]
                                axis_permutations.append(perm)

    for perm in axis_permutations:
        cos_angle = np.dot(perm, axis2)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = np.degrees(np.arccos(cos_angle))
        if angle <= angle_threshold:
            return True, perm

    return False, None

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=15.0, tolerance_widening=2):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary.
    The brandon_threshold is widened by 'tolerance_widening' factor.
    """
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
               'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }

    axis = np.array(axis)

    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            # Check axis equivalency
            eq_result, eq_axis = are_axes_equivalent(axis, predefined_axis, angle_threshold)
            if eq_result:
                predefined_angle = config['angle']
                # A second time to retrieve the "standard" version of that axis
                eq_result2, standard_axis = are_axes_equivalent(axis, predefined_axis, angle_threshold)
                deviation = abs(misorientation_angle - predefined_angle)
                brandon_threshold = tolerance_widening * 15 / np.sqrt(float(sigma.rstrip('abc')))
                if deviation <= brandon_threshold:
                    return True, sigma, deviation, standard_axis, predefined_angle

    return False, None, None, None, None


###############################################################################
# 2. Build the Tkinter GUI to accept Euler angles and show the results        #
###############################################################################
def calculate_misorientation():
    """
    Callback for the "Calculate" button.
    Reads the user input for Euler angles of grain1 and grain2,
    then performs the existing calculations, and shows the results.
    """
    try:
        # Read user-entered values
        e1_str = entry_g1.get().strip()
        e2_str = entry_g2.get().strip()
        structure_type = combo_structure.get().strip()

        # Parse into float arrays
        euler_grain1 = list(map(float, e1_str.split(',')))
        euler_grain2 = list(map(float, e2_str.split(',')))
        if len(euler_grain1) != 3 or len(euler_grain2) != 3:
            raise ValueError("Euler angles must be three comma-separated values each.")

        # Compute symmetry-reduced orientation
        euler_grain2_sym_reduced = compute_symmetry_reduced_orientation(euler_grain1, euler_grain2)

        # Calculate misorientation axis and angle
        axis, misorientation_ang = misorientation_calc(euler_grain1, euler_grain2_sym_reduced)

        # Check if it is a Σ boundary
        is_sig, sigma_value, deviation, standard_axis, predefined_angle = is_sigma_boundary(
            axis[0], misorientation_ang, structure_type=structure_type
        )

        # Build a result string
        result = []
        result.append(f"Symmetry-reduced grain2: {np.round(euler_grain2_sym_reduced[0],4)} deg")
        result.append(f"Misorientation axis (approx.): {np.round(axis[0],4)}")
        result.append(f"Misorientation angle (deg): {np.round(misorientation_ang,4)}")

        if is_sig:
            result.append(f"**This is a Σ{sigma_value} boundary**")
            result.append(f"Deviation from exact Σ boundary angle: {np.round(deviation,4)} deg")
            result.append(f"Standard axis for Σ{sigma_value}: {standard_axis}")
            result.append(f"Standard angle for Σ{sigma_value}: {predefined_angle} deg")

            # Optional: Check deviation to the perfect orientation
            # Recompute standard Euler angles for that boundary
            std_euler_angles = crystal_rotation(euler_grain1, standard_axis, predefined_angle)
            # Compare actual grain2 to standard orientation
            dev_axis, dev_angle = misorientation_calc(euler_grain2_sym_reduced, std_euler_angles)
            result.append(f"Additional deviation from exact orientation: {np.round(dev_angle,4)} deg, deviation axis: {np.round(dev_axis[0],4)}.")
        else:
            result.append("This is not a recognized low Σ boundary.")

        # Show in text box
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, "\n".join(result))

    except Exception as ex:
        messagebox.showerror("Error", str(ex))

# Create the main window
root = tk.Tk()
root.title("Grain Boundary Misorientation")

# Create and place labels, entries, and a calculate button
frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.N, tk.S, tk.E, tk.W))

label_g1 = ttk.Label(frame, text="Grain1 Euler angles (degrees, comma-separated):")
label_g1.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
entry_g1 = ttk.Entry(frame, width=30)
entry_g1.grid(row=0, column=1, padx=5, pady=5, sticky=tk.W)
entry_g1.insert(0, "294.57,46.91,40.1301")  # Example default

label_g2 = ttk.Label(frame, text="Grain2 Euler angles (degrees, comma-separated):")
label_g2.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
entry_g2 = ttk.Entry(frame, width=30)
entry_g2.grid(row=1, column=1, padx=5, pady=5, sticky=tk.W)
entry_g2.insert(0, "74.349,25.8538,282.2126")  # Example default

label_struct = ttk.Label(frame, text="Crystal Structure (for Σ data):")
label_struct.grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
combo_structure = ttk.Combobox(frame, values=["FCC", "BCC"], width=5)
combo_structure.grid(row=2, column=1, padx=5, pady=5, sticky=tk.W)
combo_structure.current(0)  # Default to "FCC"

btn_calc = ttk.Button(frame, text="Calculate", command=calculate_misorientation)
btn_calc.grid(row=3, column=0, columnspan=2, pady=10)

# Output text box
output_text = tk.Text(frame, width=60, height=15)
output_text.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

# Make the GUI responsive
for i in range(2):
    frame.columnconfigure(i, weight=1)

root.mainloop()


In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np

# For misorientation calculations
from orix.quaternion import Orientation, Quaternion, symmetry
from orix.vector import Vector3d
from scipy.spatial.transform import Rotation as R
# Simple tooltip class for tkinter widgets
def CreateToolTip(widget, text):
    """
    Create a tooltip for a given widget
    """
    tipwindow = None

    def showtip(event=None):
        nonlocal tipwindow
        if tipwindow or not text:
            return
        x = widget.winfo_rootx() + 20
        y = widget.winfo_rooty() + widget.winfo_height() + 1
        tipwindow = tw = tk.Toplevel(widget)
        tw.wm_overrideredirect(True)
        tw.wm_geometry(f"+{x}+{y}")
        label = tk.Label(
            tw,
            text=text,
            justify=tk.LEFT,
            background="#ffffe0",
            relief=tk.SOLID,
            borderwidth=1,
            font=("tahoma", "8", "normal")
        )
        label.pack(ipadx=1)

    def hidetip(event=None):
        nonlocal tipwindow
        if tipwindow:
            tipwindow.destroy()
            tipwindow = None

    widget.bind("<Enter>", showtip)
    widget.bind("<Leave>", hidetip)
    
# 1. Core functions
def rotation_matrix_to_euler(rotation_matrix):
    return rotation_matrix.as_euler('ZXZ', degrees=True)

def normalize_vector(vector):
    norm = np.linalg.norm(vector)
    if norm == 0:
        return vector
    return vector / norm

def misorientation_angle(rot1, rot2):
    delta_rot = rot1.inv() * rot2
    angle = delta_rot.magnitude()
    return np.degrees(angle)

def crystal_rotation(euler_angles, axis, angle):
    initial_rotation = R.from_euler('ZXZ', euler_angles, degrees=True)
    crystal_axis = normalize_vector(axis)
    transformed_axis = initial_rotation.apply(crystal_axis)
    angle_radians = np.radians(angle)
    axis_rotation = R.from_rotvec(angle_radians * transformed_axis)
    new_rotation = axis_rotation * initial_rotation
    new_euler_angles = rotation_matrix_to_euler(new_rotation)
    return new_euler_angles

def compute_symmetry_reduced_orientation(ori1, ori2, symmetry_str='Oh'):
    ori1_quat = Quaternion.from_euler(ori1, degrees=True)
    ori1_quat.symmetry = getattr(symmetry, symmetry_str)
    ori2_quat = Quaternion.from_euler(ori2, degrees=True)
    ori2_quat.symmetry = getattr(symmetry, symmetry_str)
    symmetry_ori = ori1_quat.symmetry

    angles = []
    ori2_sym = symmetry_ori.outer(ori2_quat)
    misorientation = ori1_quat * ~ori2_sym

    for orien in misorientation:
        orim = Orientation(orien)
        angles.append(orim.angle)

    angles_array = np.array(angles)
    indices = angles_array.argmin(axis=0)
    out = np.take_along_axis(ori2_sym, indices[np.newaxis], axis=0).squeeze()
    ori2_sym_reduced = out.to_euler(degrees=True)
    return ori2_sym_reduced

def misorientation(rot1, rot2):
    delta_rot = rot1.inv() * rot2
    misorientation_axis = delta_rot.as_rotvec()
    angle = np.degrees(delta_rot.magnitude())
    if 180 - angle < angle:
        misorientation_axis = -misorientation_axis
        angle = 180 - angle
    return misorientation_axis, angle

def misorientation_calc(euler_angles1, euler_angles2):
    rot1 = R.from_euler('ZXZ', euler_angles1, degrees=True)
    rot2 = R.from_euler('ZXZ', euler_angles2, degrees=True)
    misorientation_axis, misorientation_ang = misorientation(rot1, rot2)
    return misorientation_axis, misorientation_ang

def are_axes_equivalent(axis2, axis1, angle_threshold=5.0):
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    axis_permutations = []
    signs = [+1, -1]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if len({i, j, k}) == 3:
                    for s1 in signs:
                        for s2 in signs:
                            for s3 in signs:
                                perm = [s1 * axis1[i], s2 * axis1[j], s3 * axis1[k]]
                                axis_permutations.append(perm)
    for perm in axis_permutations:
        cos_angle = np.dot(perm, axis2)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = np.degrees(np.arccos(cos_angle))
        if angle <= angle_threshold:
            return True, perm
    return False, None

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=15.0, tolerance_widening=2):
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '15': {'FCC': [{'axis': [2, 1, 0], 'angle': 48.2}],
               'BCC': [{'axis': [2, 1, 0], 'angle': 48.2}]},
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 1], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 1], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 1, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [3, 1, 1], 'angle': 33.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 1], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [3, 3, 1], 'angle': 43.2}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [1, 0, 0], 'angle': 18.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [3, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [3, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [1, 1, 1], 'angle': 50.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 1, 1], 'angle': 32.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [3, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [3, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 0, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 1, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [1, 1, 1], 'angle': 15.2}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [2, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [3, 3, 2], 'angle': 60.8}],
                'BCC': [{'axis': [3, 3, 2], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [3, 1, 1], 'angle': 28.6}],
                'BCC': [{'axis': [3, 1, 1], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [2, 2, 1], 'angle': 53.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [3, 3, 1], 'angle': 37.1}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [3, 2, 0], 'angle': 43.7}],
                'BCC': [{'axis': [3, 2, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [1, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [5, 1, 1], 'angle': 43.6}],
                'BCC': [{'axis': [5, 1, 1], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [3, 2, 2], 'angle': 49.2}],
                'BCC': [{'axis': [3, 2, 2], 'angle': 49.2}]}
    }
    

    axis = np.array(axis)

    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            eq_result, eq_axis = are_axes_equivalent(axis, predefined_axis, angle_threshold)
            if eq_result:
                predefined_angle = config['angle']
                deviation = abs(misorientation_angle - predefined_angle)
                brandon_threshold = tolerance_widening * 15 / np.sqrt(float(sigma.rstrip('abc')))
                if deviation <= brandon_threshold:
                    return True, sigma, deviation, eq_axis, predefined_angle

    return False, None, None, None, None

# 2. Add Cube Drawing
def draw_cube(canvas, euler_angles, size=80):
    canvas.delete("all")
    w, h = canvas.winfo_width(), canvas.winfo_height()
    cx, cy = w/2, h/2
    d = size/2
    axis_len = size*0.8

    # 1) Build the Bunge ZXZ rotation matrix
    phi1, Phi, phi2 = np.deg2rad(euler_angles)
    c1, s1 = np.cos(phi1), np.sin(phi1)
    c,  s  = np.cos(Phi),  np.sin(Phi)
    c2, s2 = np.cos(phi2), np.sin(phi2)
    Rz1 = np.array([[ c1,-s1,0],[ s1, c1,0],[  0,   0,1]])
    Rx  = np.array([[  1,   0,   0],[  0,   c,  -s],[  0,   s,   c]])
    Rz2 = np.array([[ c2,-s2,0],[ s2, c2,0],[  0,   0,   1]])
    Rm  = Rz1 @ Rx @ Rz2

    # helper to project a 3D point to 2D canvas coords
    def proj(pt3):
        return (pt3[0] + cx, -pt3[1] + cy)

    # 2) Draw global frame
    canvas.create_line(cx,cy, cx+axis_len,cy,
                       fill='red', width=2, arrow=tk.LAST)
    canvas.create_text(cx+axis_len+5, cy, text='X', fill='red', anchor='w')
    canvas.create_line(cx,cy, cx,cy-axis_len,
                       fill='blue', width=2, arrow=tk.LAST)
    canvas.create_text(cx, cy-axis_len-5, text='Y', fill='blue', anchor='s')
    rdot = 4
    canvas.create_oval(cx-rdot,cy-rdot, cx+rdot,cy+rdot,
                       fill='green', outline='')
    canvas.create_text(cx+rdot+3, cy+rdot+3, text='Z', fill='green', anchor='nw')

    # 3) Legend in top-left
    legend = [
        ('[100]', 'magenta'),
        ('[010]', 'orange'),
        ('[001]', 'teal'),
    ]
    lx, ly = 10, 10
    line_len = 20
    for i, (label, color) in enumerate(legend):
        y = ly + i*20
        canvas.create_line(lx, y, lx+line_len, y, fill=color, width=3)
        canvas.create_text(lx+line_len+5, y, text=label, anchor='w')

    # 4) Compute rotated cube verts & project
    verts = np.array([
        [-d,-d,-d],[-d,-d, d],[-d, d,-d],[-d, d, d],
        [ d,-d,-d],[ d,-d, d],[ d, d,-d],[ d, d, d]
    ])
    rot_verts = verts @ Rm.T
    proj2 = [proj(p) for p in rot_verts]

    # 5) Define and draw only the three edge‐groups
    faces = [[0,1,3,2],[4,6,7,5],[0,4,5,1],
             [2,3,7,6],[0,2,6,4],[1,5,7,3]]
    normals = [np.array([-1,0,0]),np.array([1,0,0]),
               np.array([0,-1,0]),np.array([0,1,0]),
               np.array([0,0,-1]),np.array([0,0,1])]
    face_vis = [(Rm @ n)[2] > 0 for n in normals]

    edges = {
        '[100]': [(0,4),(1,5),(2,6),(3,7)],
        '[010]': [(0,2),(1,3),(4,6),(5,7)],
        '[001]': [(0,1),(2,3),(4,5),(6,7)],
    }
    colors = {'[100]':'magenta','[010]':'orange','[001]':'teal'}

    for label, eds in edges.items():
        col = colors[label]
        for i,j in eds:
            adj = [fi for fi,face in enumerate(faces) if i in face and j in face]
            front = any(face_vis[fi] for fi in adj)
            x1,y1 = proj2[i]; x2,y2 = proj2[j]
            if front:
                canvas.create_line(x1,y1,x2,y2, fill=col, width=2)
            else:
                canvas.create_line(x1,y1,x2,y2, fill=col,
                                   width=1, dash=(4,2))


# 3. GUI part
def calculate_misorientation():
    try:
        e1_str = entry_g1.get().strip()
        e2_str = entry_g2.get().strip()
        structure_type = combo_structure.get().strip()
        
                # Read new threshold inputs
        angle_threshold = float(entry_threshold.get().strip())
        tolerance_widening = float(entry_tolerance.get().strip())

        euler_grain1 = list(map(float, e1_str.split(',')))
        euler_grain2 = list(map(float, e2_str.split(',')))

        if len(euler_grain1) != 3 or len(euler_grain2) != 3:
            raise ValueError("Euler angles must be three comma-separated values each.")

        euler_grain2_sym_reduced = compute_symmetry_reduced_orientation(euler_grain1, euler_grain2)

        axis, misorientation_ang = misorientation_calc(euler_grain1, euler_grain2_sym_reduced)

        # Pass the user-specified thresholds
        is_sig, sigma_value, deviation, standard_axis, predefined_angle = is_sigma_boundary(
            axis[0], misorientation_ang,
            structure_type=structure_type,
            angle_threshold=angle_threshold,
            tolerance_widening=tolerance_widening
        )

        result = []
        result.append(f"Symmetry-reduced grain2: {np.round(euler_grain2_sym_reduced[0], 4)} deg")
        result.append(f"Misorientation axis (approx.): {np.round(axis[0], 4)}")
        result.append(f"Misorientation angle (deg): {np.round(misorientation_ang, 4)}")

        if is_sig:
            result.append(f"**This is a Σ{sigma_value} boundary**")
            result.append(f"Deviation from exact Σ boundary angle: {np.round(deviation, 4)} deg")
            result.append(f"Standard axis for Σ{sigma_value}: {standard_axis}")
            result.append(f"Standard angle for Σ{sigma_value}: {predefined_angle} deg")
                       # Optional: Check deviation to the perfect orientation
            # Recompute standard Euler angles for that boundary
            std_euler_angles = crystal_rotation(euler_grain1, standard_axis, predefined_angle)
            # Compare actual grain2 to standard orientation
            dev_axis, dev_angle = misorientation_calc(euler_grain2_sym_reduced, std_euler_angles)
            result.append(f"Additional deviation from exact orientation: {np.round(dev_angle,4)} deg, deviation axis: {np.round(dev_axis[0],4)}.")
        else:
            result.append("This is not a recognized low Σ boundary.")

            
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, "\n".join(result))

        # Update cubes
        draw_cube(canvas_g1, euler_grain1)
        draw_cube(canvas_g2, euler_grain2)

    except Exception as ex:
        messagebox.showerror("Error", str(ex))

# 3. GUI part
root = tk.Tk()
root.title("Grain Boundary Misorientation with Cubes")

frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.N, tk.S, tk.E, tk.W))

# Grain inputs
label_g1 = ttk.Label(frame, text="Grain1 Euler angles (deg, comma-separated):")
label_g1.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
entry_g1 = ttk.Entry(frame, width=30)
entry_g1.grid(row=0, column=1, padx=5, pady=5)
entry_g1.insert(0, "294.57,46.91,40.1301")

label_g2 = ttk.Label(frame, text="Grain2 Euler angles (deg, comma-separated):")
label_g2.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
entry_g2 = ttk.Entry(frame, width=30)
entry_g2.grid(row=1, column=1, padx=5, pady=5)
entry_g2.insert(0, "74.349,25.8538,282.2126")

# New threshold inputs
label_threshold = ttk.Label(frame, text="Angle threshold (deg):")
label_threshold.grid(row=2, column=0, padx=5, pady=5, sticky=tk.W)
entry_threshold = ttk.Entry(frame, width=10)
entry_threshold.grid(row=2, column=1, padx=5, pady=5, sticky=tk.W)
entry_threshold.insert(0, "10")
CreateToolTip(label_threshold, "Maximum allowed deviation angle (°) between the rotation axis and the standard axis")

label_tolerance = ttk.Label(frame, text="Tolerance widening factor:")
label_tolerance.grid(row=3, column=0, padx=5, pady=5, sticky=tk.W)
entry_tolerance = ttk.Entry(frame, width=10)
entry_tolerance.grid(row=3, column=1, padx=5, pady=5, sticky=tk.W)
entry_tolerance.insert(0, "1")
CreateToolTip(label_tolerance, "Multiplier for allowable deviation range in the Brandon criterion")


# Crystal structure
label_struct = ttk.Label(frame, text="Crystal Structure (for Σ data):")
label_struct.grid(row=4, column=0, padx=5, pady=5, sticky=tk.W)
combo_structure = ttk.Combobox(frame, values=["FCC", "BCC"], width=5)
combo_structure.grid(row=4, column=1, padx=5, pady=5, sticky=tk.W)
combo_structure.current(0)

# Calculate button
btn_calc = ttk.Button(frame, text="Calculate", command=calculate_misorientation)
btn_calc.grid(row=5, column=0, columnspan=2, pady=10)

# Canvas for cubes
canvas_frame = ttk.Frame(root, padding="10")
canvas_frame.grid(row=1, column=0)
canvas_g1 = tk.Canvas(canvas_frame, width=200, height=200, bg='white')
canvas_g2 = tk.Canvas(canvas_frame, width=200, height=200, bg='white')
canvas_g1.grid(row=0, column=0, padx=10)
canvas_g2.grid(row=0, column=1, padx=10)

# Output text box
output_text = tk.Text(root, width=70, height=15)
output_text.grid(row=2, column=0, padx=5, pady=5)

root.mainloop()


SystemExit: 0

C:\Users\x.chen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
## Euler rotation Qt version

import sys
import numpy as np
from PyQt5 import QtGui, QtCore, QtWidgets
from scipy.spatial.transform import Rotation as R

# --- basic math ---
def euler_to_rotation_matrix_deg(phi1_deg, Phi_deg, phi2_deg):
    phi1 = np.deg2rad(phi1_deg)
    Phi = np.deg2rad(Phi_deg)
    phi2 = np.deg2rad(phi2_deg)

    c1, s1 = np.cos(phi1), np.sin(phi1)
    c, s = np.cos(Phi), np.sin(Phi)
    c2, s2 = np.cos(phi2), np.sin(phi2)

    Rz1 = np.array([[c1, -s1, 0],
                    [s1,  c1, 0],
                    [ 0,   0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, c, -s],
                   [0, s, c]])
    Rz2 = np.array([[c2, -s2, 0],
                    [s2,  c2, 0],
                    [ 0,   0, 1]])
    return Rz1 @ Rx @ Rz2

def compute_wireframe_edges_visibility(R, center, size):
    d = size
    verts = np.array([
        [-d,-d,-d],[-d,-d,d],[-d,d,-d],[-d,d,d],
        [d,-d,-d],[d,-d,d],[d,d,-d],[d,d,d]
    ], float)
    rotated = verts @ R.T
    xy = rotated[:, :2]

    pts2 = np.empty_like(xy)
    pts2[:, 0] = center[0] + xy[:, 0]
    pts2[:, 1] = center[1] - xy[:, 1]

    faces = [
        [0,1,3,2], [4,6,7,5], [0,4,5,1],
        [2,3,7,6], [0,2,6,4], [1,5,7,3]
    ]
    normals = [
        np.array([-1,0,0]), np.array([1,0,0]),
        np.array([0,-1,0]), np.array([0,1,0]),
        np.array([0,0,-1]), np.array([0,0,1]),
    ]
    nz = [(R @ n)[2] for n in normals]

    edge_indices = [
        (0,1),(0,2),(0,4),(1,3),(1,5),
        (2,3),(2,6),(3,7),(4,5),(4,6),
        (5,7),(6,7)
    ]

    result = []
    for i, j in edge_indices:
        p1, p2 = pts2[i], pts2[j]
        adj = [k for k, f in enumerate(faces) if i in f and j in f]
        visible = any(nz[k] > 0 for k in adj)
        result.append((p1, p2, visible))
    return result

def axis_angle_to_rotation_matrix(axis, angle_deg):
    angle = np.deg2rad(angle_deg)
    axis = axis / np.linalg.norm(axis)
    x, y, z = axis
    c = np.cos(angle)
    s = np.sin(angle)
    C = 1 - c
    return np.array([
        [c+x*x*C, x*y*C-z*s, x*z*C+y*s],
        [y*x*C+z*s, c+y*y*C, y*z*C-x*s],
        [z*x*C-y*s, z*y*C+x*s, c+z*z*C]
    ])

def rotation_matrix_to_bunge_euler_deg(R):
    if abs(R[2,2]) < 1-1e-6:
        Phi = np.arccos(R[2,2])
        phi1 = np.arctan2(R[2,0], -R[2,1])
        phi2 = np.arctan2(R[0,2], R[1,2])
    else:
        Phi = 0 if R[2,2] > 0 else np.pi
        phi1 = np.arctan2(-R[0,1], R[0,0])
        phi2 = 0
    return np.rad2deg(phi1)%360, np.rad2deg(Phi), np.rad2deg(phi2)%360

# --- GUI ---
class CubeOverlayView(QtWidgets.QGraphicsView):
    def __init__(self, array2d, parent=None):
        super().__init__(parent)
        h, w = array2d.shape
        norm = np.clip((array2d - array2d.min())/array2d.ptp()*255, 0, 255).astype(np.uint8)
        img = QtGui.QImage(norm.data, w, h, w, QtGui.QImage.Format_Grayscale8)
        pix = QtGui.QPixmap.fromImage(img)

        self.scene = QtWidgets.QGraphicsScene(self)
        self.setScene(self.scene)
        self.pix_item = QtWidgets.QGraphicsPixmapItem(pix)
        self.scene.addItem(self.pix_item)
        self.setRenderHint(QtGui.QPainter.Antialiasing)

    def overlay_cube_and_axes(self, R, size=40):
        for item in list(self.scene.items()):
            if isinstance(item, (QtWidgets.QGraphicsLineItem, QtWidgets.QGraphicsTextItem)):
                self.scene.removeItem(item)

        W, H = self.pix_item.pixmap().width(), self.pix_item.pixmap().height()
        cx, cy = W/2, H/2
        center = (cx, cy)
        axis_len = size * 1.5
        text_offset = 10

        # X axis
        x_line = QtWidgets.QGraphicsLineItem(cx, cy, cx + axis_len, cy)
        pen_x = QtGui.QPen(QtGui.QColor(255,0,0)); pen_x.setWidth(2)
        x_line.setPen(pen_x)
        self.scene.addItem(x_line)
        txt_x = QtWidgets.QGraphicsTextItem("X")
        txt_x.setDefaultTextColor(QtGui.QColor(255,0,0))
        txt_x.setFont(QtGui.QFont("Arial", 12))
        txt_x.setPos(cx + axis_len + text_offset, cy - text_offset)
        self.scene.addItem(txt_x)

        # Y axis
        y_line = QtWidgets.QGraphicsLineItem(cx, cy, cx, cy - axis_len)
        pen_y = QtGui.QPen(QtGui.QColor(0,0,255)); pen_y.setWidth(2)
        y_line.setPen(pen_y)
        self.scene.addItem(y_line)
        txt_y = QtWidgets.QGraphicsTextItem("Y")
        txt_y.setDefaultTextColor(QtGui.QColor(0,0,255))
        txt_y.setFont(QtGui.QFont("Arial", 12))
        txt_y.setPos(cx + text_offset, cy - axis_len - text_offset)
        self.scene.addItem(txt_y)

        # cube edges
        for (p1, p2, visible) in compute_wireframe_edges_visibility(R, center, size):
            (x1, y1), (x2, y2) = p1, p2
            pen = QtGui.QPen(QtGui.QColor(0,255,0))
            if visible:
                pen.setWidth(2)
                pen.setStyle(QtCore.Qt.SolidLine)
            else:
                pen.setWidth(1)
                pen.setStyle(QtCore.Qt.DashLine)
            line = QtWidgets.QGraphicsLineItem(x1, y1, x2, y2)
            line.setPen(pen)
            self.scene.addItem(line)

class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        data = np.random.normal(100,30,(300,300))

        central = QtWidgets.QWidget()
        self.setCentralWidget(central)
        vbox = QtWidgets.QVBoxLayout(central)

        self.view = CubeOverlayView(data)
        vbox.addWidget(self.view, stretch=1)

        controls = QtWidgets.QFormLayout()
        vbox.addLayout(controls)

        self.spin_phi1 = QtWidgets.QDoubleSpinBox(); self.spin_phi1.setRange(0,360); self.spin_phi1.setValue(30)
        self.spin_Phi  = QtWidgets.QDoubleSpinBox(); self.spin_Phi.setRange(0,180); self.spin_Phi.setValue(45)
        self.spin_phi2 = QtWidgets.QDoubleSpinBox(); self.spin_phi2.setRange(0,360); self.spin_phi2.setValue(60)
        controls.addRow("φ₁ (°)", self.spin_phi1)
        controls.addRow("Φ (°)",  self.spin_Phi)
        controls.addRow("φ₂ (°)", self.spin_phi2)

        self.spin_ax_x = QtWidgets.QDoubleSpinBox(); self.spin_ax_x.setRange(-1,1); self.spin_ax_x.setValue(0)
        self.spin_ax_y = QtWidgets.QDoubleSpinBox(); self.spin_ax_y.setRange(-1,1); self.spin_ax_y.setValue(0)
        self.spin_ax_z = QtWidgets.QDoubleSpinBox(); self.spin_ax_z.setRange(-1,1); self.spin_ax_z.setValue(1)
        self.spin_ax_angle = QtWidgets.QDoubleSpinBox(); self.spin_ax_angle.setRange(-360,360); self.spin_ax_angle.setValue(0)

        controls.addRow("Axis X", self.spin_ax_x)
        controls.addRow("Axis Y", self.spin_ax_y)
        controls.addRow("Axis Z", self.spin_ax_z)
        controls.addRow("Rotate (°)", self.spin_ax_angle)

        self.label_rotated = QtWidgets.QLabel("Rotated: φ₁=0.0°, Φ=0.0°, φ₂=0.0°")
        vbox.addWidget(self.label_rotated)

        # --- 保存槽 ---
        self.saved_configs = [None]*4
        slots_layout = QtWidgets.QGridLayout()
        vbox.addLayout(slots_layout)

        # Save buttons on 0,2 rows; Load buttons on 1,3 rows
        for i in range(4):
            btn_save = QtWidgets.QPushButton(f"Save {i+1}")
            btn_load = QtWidgets.QPushButton(f"Load {i+1}")
            btn_save.clicked.connect(lambda checked, idx=i: self.save_to_slot(idx))
            btn_load.clicked.connect(lambda checked, idx=i: self.load_from_slot(idx))

            if i < 4:
                row_save = 0
                row_load = 1
                col = i
            else:
                row_save = 2
                row_load = 3
                col = i - 4

            slots_layout.addWidget(btn_save, row_save, col)
            slots_layout.addWidget(btn_load, row_load, col)


        # spinbox 绑定
        for spin in (self.spin_phi1, self.spin_Phi, self.spin_phi2):
            spin.valueChanged.connect(lambda: self.update_overlay(update_spinbox=False))
        for spin in (self.spin_ax_x, self.spin_ax_y, self.spin_ax_z, self.spin_ax_angle):
            spin.valueChanged.connect(lambda: self.update_overlay(update_spinbox=True))

        self.resize(800, 800)
        self.update_overlay()

    def update_overlay(self, update_spinbox=False):
        phi1 = self.spin_phi1.value()
        Phi  = self.spin_Phi.value()
        phi2 = self.spin_phi2.value()

        base_R = euler_to_rotation_matrix_deg(phi1, Phi, phi2)

        axis = np.array([self.spin_ax_x.value(),
                         self.spin_ax_y.value(),
                         self.spin_ax_z.value()])
        angle = self.spin_ax_angle.value()

        if np.linalg.norm(axis) > 1e-6 and abs(angle) > 1e-6:
            rot = axis_angle_to_rotation_matrix(axis, angle)
            final_R = rot @ base_R
        else:
            final_R = base_R

        self.view.overlay_cube_and_axes(final_R, size=50)
        
        ##orix and rotation_matrix_to_bunge_euler_deg don't give right results
        #phi1_new, Phi_new, phi2_new = rotation_matrix_to_bunge_euler_deg(final_R)
        #self.label_rotated.setText(f"Rotated: φ₁={phi1_new:.1f}°, Φ={Phi_new:.1f}°, φ₂={phi2_new:.1f}°")
        ##ori_R = Orientation.from_matrix(final_R)
        ##phi1_new, Phi_new, phi2_new = ori_R.to_euler(degrees=True)[0]
        ##self.label_rotated.setText(f"Rotated: φ₁={phi1_new:.1f}°, Φ={Phi_new:.1f}°, φ₂={phi2_new:.1f}°")
        rot_scipy = R.from_matrix(final_R)
        phi1_new, Phi_new, phi2_new = rot_scipy.as_euler('ZXZ', degrees=True)
        self.label_rotated.setText(f"Rotated: φ₁={phi1_new:.1f}°, Φ={Phi_new:.1f}°, φ₂={phi2_new:.1f}°")

    def save_to_slot(self, idx):
        self.saved_configs[idx] = (
            self.spin_phi1.value(), self.spin_Phi.value(), self.spin_phi2.value(),
            self.spin_ax_x.value(), self.spin_ax_y.value(), self.spin_ax_z.value(), self.spin_ax_angle.value()
        )

    def load_from_slot(self, idx):
        cfg = self.saved_configs[idx]
        if cfg is None:
            return
        phi1, Phi, phi2, axx, axy, axz, ang = cfg
        for spin, val in zip(
            (self.spin_phi1, self.spin_Phi, self.spin_phi2,
             self.spin_ax_x, self.spin_ax_y, self.spin_ax_z, self.spin_ax_angle),
            (phi1, Phi, phi2, axx, axy, axz, ang)
        ):
            spin.blockSignals(True)
            spin.setValue(val)
            spin.blockSignals(False)
        self.update_overlay()

if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    win = MainWindow()
    win.show()
    sys.exit(app.exec_())


In [ ]:
## Euler rotation TK version

import sys
import numpy as np
from tkinter import Tk, Canvas, Frame, Label, Button, Spinbox, StringVar
from scipy.spatial.transform import Rotation as SciRot

# --- basic math ---
def euler_to_rotation_matrix_deg(phi1_deg, Phi_deg, phi2_deg):
    phi1 = np.deg2rad(phi1_deg)
    Phi = np.deg2rad(Phi_deg)
    phi2 = np.deg2rad(phi2_deg)

    c1, s1 = np.cos(phi1), np.sin(phi1)
    c, s = np.cos(Phi), np.sin(Phi)
    c2, s2 = np.cos(phi2), np.sin(phi2)

    Rz1 = np.array([[c1, -s1, 0],
                    [s1,  c1, 0],
                    [ 0,   0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, c, -s],
                   [0, s,  c]])
    Rz2 = np.array([[c2, -s2, 0],
                    [s2,  c2, 0],
                    [ 0,   0, 1]])
    return Rz1 @ Rx @ Rz2


def compute_wireframe_edges_visibility(R, center, size):
    d = size
    verts = np.array([
        [-d,-d,-d],[-d,-d, d],[-d, d,-d],[-d, d, d],
        [ d,-d,-d],[ d,-d, d],[ d, d,-d],[ d, d, d]
    ], float)
    rotated = verts @ R.T
    xy = rotated[:, :2]
    pts2 = [(center[0] + x, center[1] - y) for x, y in xy]

    faces = [
        [0,1,3,2], [4,6,7,5], [0,4,5,1],
        [2,3,7,6], [0,2,6,4], [1,5,7,3]
    ]
    normals = [
        np.array([-1,0,0]), np.array([1,0,0]),
        np.array([0,-1,0]), np.array([0,1,0]),
        np.array([0,0,-1]), np.array([0,0,1]),
    ]
    nz = [(R @ n)[2] for n in normals]

    edge_indices = [
        (0,1),(0,2),(0,4),(1,3),(1,5),
        (2,3),(2,6),(3,7),(4,5),(4,6),
        (5,7),(6,7)
    ]

    result = []
    for i, j in edge_indices:
        p1, p2 = pts2[i], pts2[j]
        adj = [k for k, f in enumerate(faces) if i in f and j in f]
        visible = any(nz[k] > 0 for k in adj)
        result.append((p1, p2, visible))
    return result


def axis_angle_to_rotation_matrix(axis, angle_deg):
    angle = np.deg2rad(angle_deg)
    axis = axis / np.linalg.norm(axis)
    x, y, z = axis
    c = np.cos(angle)
    s = np.sin(angle)
    C = 1 - c
    return np.array([
        [c + x*x*C,    x*y*C - z*s, x*z*C + y*s],
        [y*x*C + z*s,  c + y*y*C,   y*z*C - x*s],
        [z*x*C - y*s,  z*y*C + x*s, c + z*z*C]
    ])


def rotation_matrix_to_bunge_euler_deg(R):
    if abs(R[2,2]) < 1 - 1e-6:
        Phi = np.arccos(R[2,2])
        phi1 = np.arctan2(R[2,0], -R[2,1])
        phi2 = np.arctan2(R[0,2], R[1,2])
    else:
        Phi = 0 if R[2,2] > 0 else np.pi
        phi1 = np.arctan2(-R[0,1], R[0,0])
        phi2 = 0
    return np.rad2deg(phi1) % 360, np.rad2deg(Phi), np.rad2deg(phi2) % 360

# --- GUI ---
class Sped3DTkinter:
    def __init__(self, master):
        self.master = master
        master.title("SPED3D - Tkinter")
        master.configure(bg='white')
        master.geometry("800x800")

        # canvas with white background
        self.canvas = Canvas(master, width=400, height=400, bg='white', highlightthickness=0)
        self.canvas.pack(padx=20, pady=20)

        # control panel
        ctrl_frame = Frame(master, bg='white')
        ctrl_frame.pack(pady=10)

        # Euler inputs
        Label(ctrl_frame, text="φ₁ (°)", bg='white').grid(row=0, column=0)
        self.spin_phi1 = Spinbox(ctrl_frame, from_=0, to=360, width=5, command=self.update_overlay)
        self.spin_phi1.grid(row=0, column=1)
        Label(ctrl_frame, text="Φ (°)", bg='white').grid(row=0, column=2)
        self.spin_Phi = Spinbox(ctrl_frame, from_=0, to=180, width=5, command=self.update_overlay)
        self.spin_Phi.grid(row=0, column=3)
        Label(ctrl_frame, text="φ₂ (°)", bg='white').grid(row=0, column=4)
        self.spin_phi2 = Spinbox(ctrl_frame, from_=0, to=360, width=5, command=self.update_overlay)
        self.spin_phi2.grid(row=0, column=5)

        # axis-angle inputs
        Label(ctrl_frame, text="Axis X", bg='white').grid(row=1, column=0)
        self.spin_ax_x = Spinbox(ctrl_frame, from_=-1, to=1, increment=0.1, width=5, command=self.update_overlay)
        self.spin_ax_x.grid(row=1, column=1)
        Label(ctrl_frame, text="Axis Y", bg='white').grid(row=1, column=2)
        self.spin_ax_y = Spinbox(ctrl_frame, from_=-1, to=1, increment=0.1, width=5, command=self.update_overlay)
        self.spin_ax_y.grid(row=1, column=3)
        Label(ctrl_frame, text="Axis Z", bg='white').grid(row=1, column=4)
        self.spin_ax_z = Spinbox(ctrl_frame, from_=-1, to=1, increment=0.1, width=5, command=self.update_overlay)
        self.spin_ax_z.grid(row=1, column=5)
        Label(ctrl_frame, text="Rotate (°)", bg='white').grid(row=2, column=0)
        self.spin_ax_angle = Spinbox(ctrl_frame, from_=-360, to=360, width=5, command=self.update_overlay)
        self.spin_ax_angle.grid(row=2, column=1)

        # set default axis-angle values
        self.spin_ax_x.delete(0, 'end'); self.spin_ax_x.insert(0, '1')
        self.spin_ax_y.delete(0, 'end'); self.spin_ax_y.insert(0, '0')
        self.spin_ax_z.delete(0, 'end'); self.spin_ax_z.insert(0, '0')
        self.spin_ax_angle.delete(0, 'end'); self.spin_ax_angle.insert(0, '0')

        # rotated label
        self.var_rot = StringVar()
        self.var_rot.set("Rotated: φ₁=0.0°, Φ=0.0°, φ₂=0.0°")
        Label(master, textvariable=self.var_rot, bg='white').pack()

        # save/load slots
        self.saved = [None]*4
        slot_frame = Frame(master, bg='white')
        slot_frame.pack(pady=5)
        for i in range(4):
            btn_save = Button(slot_frame,
                              text=f"Save {i+1}",
                              command=lambda i=i: self.save_slot(i),
                              bg='white')
            btn_save.grid(row=0, column=i, padx=5)
            btn_load = Button(slot_frame,
                              text=f"Load {i+1}",
                              command=lambda i=i: self.load_slot(i),
                              bg='white')
            btn_load.grid(row=1, column=i, padx=5)

        self.update_overlay()

    def update_overlay(self):
        self.canvas.delete('overlay')
        cx, cy = 200, 200
        size = 80
        phi1 = float(self.spin_phi1.get())
        Phi = float(self.spin_Phi.get())
        phi2 = float(self.spin_phi2.get())
        R = euler_to_rotation_matrix_deg(phi1, Phi, phi2)
        ax = np.array([float(self.spin_ax_x.get()), float(self.spin_ax_y.get()), float(self.spin_ax_z.get())])
        ang = float(self.spin_ax_angle.get())
        if np.linalg.norm(ax) > 1e-6 and abs(ang) > 1e-6:
            R = axis_angle_to_rotation_matrix(ax, ang) @ R
        length = size * 1.5
        self.canvas.create_line(cx, cy, cx+length, cy, fill='red', width=2, tags='overlay')
        self.canvas.create_text(cx+length+10, cy-10, text='X', fill='red', tags='overlay')
        self.canvas.create_line(cx, cy, cx, cy-length, fill='blue', width=2, tags='overlay')
        self.canvas.create_text(cx+10, cy-length-10, text='Y', fill='blue', tags='overlay')
        for p1, p2, vis in compute_wireframe_edges_visibility(R, (cx, cy), size):
            x1, y1 = p1; x2, y2 = p2
            dash = () if vis else (3,)
            w = 2 if vis else 1
            self.canvas.create_line(x1, y1, x2, y2, fill='green', width=w, dash=dash, tags='overlay')
        rot_scipy = SciRot.from_matrix(R)
        phi1_new, Phi_new, phi2_new = rot_scipy.as_euler('ZXZ', degrees=True)
        self.var_rot.set(f"Rotated: φ₁={phi1_new:.1f}°, Φ={Phi_new:.1f}°, φ₂={phi2_new:.1f}°")

    def save_slot(self, idx):
        self.saved[idx] = (
            float(self.spin_phi1.get()), float(self.spin_Phi.get()), float(self.spin_phi2.get()),
            float(self.spin_ax_x.get()), float(self.spin_ax_y.get()), float(self.spin_ax_z.get()), float(self.spin_ax_angle.get())
        )

    def load_slot(self, idx):
        cfg = self.saved[idx]
        if cfg is None:
            return
        phi1, Phi, phi2, axx, axy, axz, ang = cfg
        for spin, val in zip(
            (self.spin_phi1, self.spin_Phi, self.spin_phi2, self.spin_ax_x, self.spin_ax_y, self.spin_ax_z, self.spin_ax_angle),
            (phi1, Phi, phi2, axx, axy, axz, ang)
        ):
            spin.delete(0, 'end'); spin.insert(0, str(val))
        self.update_overlay()

if __name__ == '__main__':
    root = Tk()
    app = Sped3DTkinter(root)
    root.mainloop()


C:\Users\x.chen\AppData\Local\Temp\ipykernel_39580\16403977.py:184: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  phi1_new, Phi_new, phi2_new = rot_scipy.as_euler('ZXZ', degrees=True)
